In [68]:
#import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()

In [69]:
#import data
xls_file = pd.ExcelFile('C:\\Users\\sonja tilly.ABERDEEN\\credit spread drivers 0717.xlsx')

**Prep data**

In [70]:
#parse sheet
mthly = xls_file.parse('monthly_transformed')
mthly = mthly.fillna(0.0000001)
mthly.tail(1)

,FDTR Index,VIX Index,USTW$ Index,USGG10YR Index,USGG30YR Index,USGG5YR Index,USYC2Y10 Index,USYC2Y30 Index,USYC5Y10 Index,USYC2Y5Y Index,...,CICRTOT Index,ARDIMOYY Index,M1% YOY Index,SCGRRAI Index,EMB US Equity,SPE AUTO Index,SPE CARD Index,BFCIUS Index,INJCJMOM Index,LEI CHNG Index
NaT,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,...,1.000000e-07,1.000000e-07,0.0,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07


In [71]:
#parse sheet
qly = xls_file.parse('qly1_transformed')
qly = qly.fillna(0.0000001)
qly.head(2)

,GDP CYOY Index
1997-03-31,1.000000e-07
1997-06-30,-6.976744e-02


In [72]:
#parse sheet
qly2 = xls_file.parse('qly2_transformed')
qly2 = qly2.fillna(0.0000001)
qly2.head(1)

,SLDETIGT Index,SLDETGTS Index
1997-01-31,1.000000e-07,1.000000e-07


In [73]:
#parse sheet
dly = xls_file.parse('dly_transformed')
dly = qly2.fillna(0.0000001)
dly.head(1)

,SLDETIGT Index,SLDETGTS Index
1997-01-31,1.000000e-07,1.000000e-07


In [74]:
upsampled = qly.resample('D').mean().fillna(method='bfill')
upsampled.head()

,GDP CYOY Index
1997-03-31,1.000000e-07
1997-04-01,-6.976744e-02
1997-04-02,-6.976744e-02
1997-04-03,-6.976744e-02
1997-04-04,-6.976744e-02


In [75]:
upsampled2 = qly2.resample('D').mean().fillna(method='bfill')
upsampled2.head()

,SLDETIGT Index,SLDETGTS Index
1997-01-31,1.000000e-07,1.000000e-07
1997-02-01,2.142857e-01,-5.142857e-01
1997-02-02,2.142857e-01,-5.142857e-01
1997-02-03,2.142857e-01,-5.142857e-01
1997-02-04,2.142857e-01,-5.142857e-01


In [76]:
#upsampled3 = dly.resample('M').mean().fillna(method='bfill')
#upsampled3.head()

In [77]:
#merge dataframes

merged = pd.concat([mthly, upsampled, upsampled2, dly], axis=1)
df = merged.resample('M').mean().fillna(method='bfill')

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 247 entries, 1997-01-31 to 2017-07-31
Data columns (total 47 columns):
FDTR Index        247 non-null float64
VIX Index         247 non-null float64
USTW$ Index       247 non-null float64
USGG10YR Index    247 non-null float64
USGG30YR Index    247 non-null float64
USGG5YR Index     247 non-null float64
USYC2Y10 Index    247 non-null float64
USYC2Y30 Index    247 non-null float64
USYC5Y10 Index    247 non-null float64
USYC2Y5Y Index    247 non-null float64
CPI YOY Index     247 non-null float64
FDIDFDMO Index    247 non-null float64
USURTOT Index     247 non-null float64
NFP TCH Index     247 non-null float64
USEMNCHG Index    247 non-null float64
IP CHNG Index     247 non-null float64
MTIBCHNG Index    247 non-null float64
MGT2TB Index      247 non-null float64
PIDSPINX Index    247 non-null float64
NAPMPMI Index     247 non-null float64
OUTFGAF Index     247 non-null float64
USGGBE10 Index    247 non-null float64
USHBMIDX Index    

C:\Users\sonja tilly.ABERDEEN\AppData\Local\Continuum\Anaconda2\envs\py35\lib\site-packages\pandas\indexes\base.py:1877: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


**Create new features**

In [80]:
#calculate rolling 3M z-score

cols = list(df.columns)

for col in cols:
    col_zscore = col + '_zscore'
    mean = df[col].rolling(center=False, window=3).mean()
    std = df[col].rolling(center=False, window=3).std()
    df[col_zscore] = (df[col] - mean)/std


ValueError: Wrong number of items passed 2, placement implies 1

In [ ]:
#address data lags

df[['CPI YOY Index','FDIDFDMO Index','USURTOT Index','NFP TCH Index', 'USEMNCHG Index',\
   'IP CHNG Index', 'MTIBCHNG Index', 'MGT2TB Index', 'OUTFGAF Index', 'USHBMIDX Index',\
   'CONCCONF Index','RSTAMOM Index', 'USTBTOT Index', 'CONSSENT Index', 'CICRTOT Index',\
   'SPE AUTO Index', 'SPE CARD Index', 'GDP CYOY Index']].shift(3).fillna(0.0000001)

df.head()

In [ ]:
#parse sheet
tgt = xls_file.parse('tgt')
y = np.array(tgt).ravel()

In [ ]:
# For each variable in X, calculate VIF and save in dataframe

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif["features"] = df.columns
vif = (vif[vif.vif >= 5])
vif

In [ ]:
#drop features with infinite or above 5 variance inflation factor

X.drop(X[['USGG10YR% Index_zscore', 'USGG30YR% Index_zscore', 'USGG5YR% Index_zscore',\
                  'USTBTOT% Index_zscore', 'GDPCTOT% Index_zscore',\
                  'GPDITOC% Index_zscore', 'GDP CQOQ Index_zscore']], axis=1, inplace=True)

In [ ]:
#split dataset

X_train = X[:196]
y_train = y[:196]
X_test = X[196:]
y_test = y[196:]
X_train.head()

In [ ]:
#try Lasso regression

from sklearn import linear_model

lasso = linear_model.Lasso(alpha=0.21, fit_intercept=False, normalize=False, precompute=False,\
                           copy_X=True, max_iter=1000, tol=0.01, warm_start=False,\
                           positive=False, random_state=42, selection='cyclic')
lasso.fit(X_train, y_train)
lasso.score(X_test,y_test)

In [ ]:
#try RandomForests

from sklearn.ensemble import RandomForestRegressor

tree = RandomForestRegressor(n_estimators=30, criterion='mse',
                            max_depth=4, min_samples_split=2, 
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.30,
                            max_features='auto',
                            max_leaf_nodes=None,
                            min_impurity_split=1e-07,
                            bootstrap=True, oob_score=False, n_jobs=-1,
                            random_state=None, verbose=0,
                            warm_start=False)
tree.fit(X_train, y_train)
tree.score(X_test, y_test)